# Module 2 Lab: Big-6 SQL Statements Mastery

Master the six essential SQL statements that form the foundation of database operations.

## Learning Objectives
- Master SELECT queries with complex conditions
- Implement secure INSERT operations
- Perform safe UPDATE and DELETE operations
- Create robust database schemas with CREATE TABLE
- Manage schema cleanup with DROP TABLE
- Apply best practices and optimization techniques

**Estimated Time**: 120-150 minutes

In [ ]:
import sqlite3
import json
import time
from datetime import datetime, timedelta
from decimal import Decimal
import random

# Create database connection
conn = sqlite3.connect('lab2_big_six.db')
conn.row_factory = sqlite3.Row

# Apply optimizations
conn.execute("PRAGMA foreign_keys = ON")
conn.execute("PRAGMA journal_mode = WAL")

print("✅ Database connection established for Big-6 SQL Lab")

## Section 1: CREATE TABLE - Schema Design

In [ ]:
# Exercise 1.1: Create comprehensive e-commerce schema

def create_ecommerce_schema(conn):
    """Create complete e-commerce database schema"""
    
    cursor = conn.cursor()
    
    # Categories table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            description TEXT,
            parent_id INTEGER REFERENCES categories(id),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            CHECK(length(trim(name)) > 0)
        )
    """)
    
    # Customers table with comprehensive validation
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL CHECK(length(trim(first_name)) > 0),
            last_name TEXT NOT NULL CHECK(length(trim(last_name)) > 0),
            email TEXT NOT NULL UNIQUE CHECK(email LIKE '%@%.%'),
            phone TEXT,
            date_of_birth DATE,
            address JSON,
            customer_type TEXT DEFAULT 'regular' CHECK(customer_type IN ('regular', 'premium', 'vip')),
            credit_limit DECIMAL(10,2) DEFAULT 1000.00 CHECK(credit_limit >= 0),
            is_active BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Products table with rich metadata
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sku TEXT NOT NULL UNIQUE,
            name TEXT NOT NULL CHECK(length(trim(name)) > 0),
            description TEXT,
            category_id INTEGER NOT NULL REFERENCES categories(id),
            price DECIMAL(10,2) NOT NULL CHECK(price >= 0),
            cost DECIMAL(10,2) CHECK(cost >= 0),
            stock_quantity INTEGER DEFAULT 0 CHECK(stock_quantity >= 0),
            reorder_level INTEGER DEFAULT 10,
            weight DECIMAL(8,3),
            dimensions JSON,
            specifications JSON,
            is_active BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Orders table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS orders (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_number TEXT NOT NULL UNIQUE,
            customer_id INTEGER NOT NULL REFERENCES customers(id),
            order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            status TEXT DEFAULT 'pending' CHECK(status IN ('pending', 'confirmed', 'processing', 'shipped', 'delivered', 'cancelled')),
            subtotal DECIMAL(10,2) DEFAULT 0 CHECK(subtotal >= 0),
            tax_amount DECIMAL(10,2) DEFAULT 0 CHECK(tax_amount >= 0),
            shipping_amount DECIMAL(10,2) DEFAULT 0 CHECK(shipping_amount >= 0),
            total_amount DECIMAL(10,2) GENERATED ALWAYS AS (subtotal + tax_amount + shipping_amount) STORED,
            shipping_address JSON,
            notes TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Order items table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS order_items (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER NOT NULL REFERENCES orders(id) ON DELETE CASCADE,
            product_id INTEGER NOT NULL REFERENCES products(id),
            quantity INTEGER NOT NULL CHECK(quantity > 0),
            unit_price DECIMAL(10,2) NOT NULL CHECK(unit_price >= 0),
            discount_amount DECIMAL(10,2) DEFAULT 0 CHECK(discount_amount >= 0),
            line_total DECIMAL(10,2) GENERATED ALWAYS AS ((unit_price * quantity) - discount_amount) STORED,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Create indexes for performance
    indexes = [
        "CREATE INDEX IF NOT EXISTS idx_customers_email ON customers(email)",
        "CREATE INDEX IF NOT EXISTS idx_customers_type_active ON customers(customer_type, is_active)",
        "CREATE INDEX IF NOT EXISTS idx_products_category ON products(category_id)",
        "CREATE INDEX IF NOT EXISTS idx_products_sku ON products(sku)",
        "CREATE INDEX IF NOT EXISTS idx_products_active_stock ON products(is_active, stock_quantity)",
        "CREATE INDEX IF NOT EXISTS idx_orders_customer ON orders(customer_id)",
        "CREATE INDEX IF NOT EXISTS idx_orders_date_status ON orders(order_date, status)",
        "CREATE INDEX IF NOT EXISTS idx_order_items_order ON order_items(order_id)",
        "CREATE INDEX IF NOT EXISTS idx_order_items_product ON order_items(product_id)"
    ]
    
    for index in indexes:
        cursor.execute(index)
    
    conn.commit()
    print("✅ Complete e-commerce schema created with indexes")

create_ecommerce_schema(conn)

## Section 2: INSERT - Data Population

In [ ]:
# Exercise 2.1: Strategic data insertion with various patterns

def populate_categories(cursor):
    """Insert product categories with hierarchy"""
    
    # Root categories
    root_categories = [
        ('Electronics', 'Electronic devices and gadgets'),
        ('Books', 'Physical and digital books'),
        ('Clothing', 'Apparel and accessories'),
        ('Home & Garden', 'Home improvement and garden supplies'),
        ('Sports & Outdoors', 'Sports equipment and outdoor gear')
    ]
    
    cursor.executemany("""
        INSERT INTO categories (name, description) VALUES (?, ?)
    """, root_categories)
    
    # Sub-categories (using parent_id)
    sub_categories = [
        ('Computers', 'Desktop and laptop computers', 1),
        ('Smartphones', 'Mobile phones and accessories', 1),
        ('Fiction', 'Fiction books and novels', 2),
        ('Technical', 'Programming and technical books', 2),
        ('Men\'s Clothing', 'Clothing for men', 3),
        ('Women\'s Clothing', 'Clothing for women', 3)
    ]
    
    cursor.executemany("""
        INSERT INTO categories (name, description, parent_id) VALUES (?, ?, ?)
    """, sub_categories)
    
    print(f"✅ Inserted {len(root_categories)} root and {len(sub_categories)} sub-categories")

def populate_customers(cursor):
    """Insert diverse customer data"""
    
    customers_data = [
        ('John', 'Doe', 'john.doe@email.com', '+1-555-0101', '1985-03-15',
         {'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zip': '10001'}, 
         'premium', 5000.00),
        ('Jane', 'Smith', 'jane.smith@email.com', '+1-555-0102', '1990-07-22',
         {'street': '456 Oak Ave', 'city': 'Los Angeles', 'state': 'CA', 'zip': '90210'}, 
         'vip', 10000.00),
        ('Bob', 'Johnson', 'bob.johnson@email.com', '+1-555-0103', '1982-11-08',
         {'street': '789 Pine St', 'city': 'Chicago', 'state': 'IL', 'zip': '60601'}, 
         'regular', 2000.00),
        ('Alice', 'Williams', 'alice.williams@email.com', '+1-555-0104', '1988-04-12',
         {'street': '321 Elm Dr', 'city': 'Houston', 'state': 'TX', 'zip': '77001'}, 
         'premium', 7500.00),
        ('Charlie', 'Brown', 'charlie.brown@email.com', '+1-555-0105', '1995-09-30',
         {'street': '654 Maple Ln', 'city': 'Phoenix', 'state': 'AZ', 'zip': '85001'}, 
         'regular', 1500.00)
    ]
    
    cursor.executemany("""
        INSERT INTO customers (first_name, last_name, email, phone, date_of_birth, 
                             address, customer_type, credit_limit)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, customers_data)
    
    print(f"✅ Inserted {len(customers_data)} customers")

def populate_products(cursor):
    """Insert products with rich metadata"""
    
    products_data = [
        # Electronics - Computers
        ('LAPTOP-001', 'Gaming Laptop Pro', 'High-performance gaming laptop', 6, 1299.99, 800.00, 15, 5,
         2.5, {'length': 35, 'width': 25, 'height': 2}, 
         {'cpu': 'Intel i7', 'ram': '16GB', 'storage': '512GB SSD', 'gpu': 'RTX 3070'}),
        ('DESKTOP-001', 'Workstation Desktop', 'Professional workstation', 6, 1899.99, 1200.00, 8, 3,
         8.0, {'length': 45, 'width': 20, 'height': 40},
         {'cpu': 'Intel i9', 'ram': '32GB', 'storage': '1TB SSD', 'gpu': 'RTX 3080'}),
        
        # Electronics - Smartphones  
        ('PHONE-001', 'SmartPhone X', 'Latest flagship smartphone', 7, 899.99, 500.00, 25, 10,
         0.2, {'length': 15, 'width': 7, 'height': 1},
         {'screen': '6.5 inch', 'storage': '256GB', 'camera': '48MP', 'battery': '4000mAh'}),
        
        # Books - Fiction
        ('BOOK-001', 'Mystery Novel', 'Bestselling mystery thriller', 8, 19.99, 8.00, 50, 20,
         0.3, {'length': 20, 'width': 13, 'height': 2},
         {'author': 'Famous Author', 'pages': 350, 'publisher': 'Great Books Inc'}),
        
        # Books - Technical
        ('BOOK-002', 'Python Programming Guide', 'Complete Python programming reference', 9, 49.99, 20.00, 30, 15,
         0.8, {'length': 25, 'width': 18, 'height': 4},
         {'author': 'Tech Expert', 'pages': 800, 'level': 'intermediate', 'edition': '3rd'})
    ]
    
    cursor.executemany("""
        INSERT INTO products (sku, name, description, category_id, price, cost, 
                            stock_quantity, reorder_level, weight, dimensions, specifications)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, products_data)
    
    print(f"✅ Inserted {len(products_data)} products with rich metadata")

# Execute all insertions
cursor = conn.cursor()
populate_categories(cursor)
populate_customers(cursor)
populate_products(cursor)
conn.commit()

print("\n✅ All initial data populated successfully")

## Section 3: SELECT - Advanced Query Mastery

In [ ]:
# Exercise 3.1: Complex SELECT queries

def demonstrate_select_patterns():
    """Demonstrate various SELECT patterns and techniques"""
    
    cursor = conn.cursor()
    
    print("SELECT Query Demonstrations:")
    print("=" * 50)
    
    # 1. Basic SELECT with conditions
    print("\n1. Premium customers with high credit limits:")
    cursor.execute("""
        SELECT first_name || ' ' || last_name as full_name,
               email, customer_type, credit_limit
        FROM customers 
        WHERE customer_type IN ('premium', 'vip') 
        AND credit_limit > 5000
        ORDER BY credit_limit DESC
    """)
    
    for row in cursor.fetchall():
        print(f"  {row['full_name']} ({row['customer_type']}) - ${row['credit_limit']}")
    
    # 2. JOIN queries
    print("\n2. Products with category information:")
    cursor.execute("""
        SELECT p.name as product_name, 
               p.price, p.stock_quantity,
               c.name as category_name,
               CASE 
                   WHEN p.stock_quantity <= p.reorder_level THEN 'Low Stock'
                   WHEN p.stock_quantity > p.reorder_level * 3 THEN 'High Stock'
                   ELSE 'Normal Stock'
               END as stock_status
        FROM products p
        JOIN categories c ON p.category_id = c.id
        WHERE p.is_active = 1
        ORDER BY c.name, p.price DESC
    """)
    
    for row in cursor.fetchall():
        print(f"  {row['product_name']} - ${row['price']} [{row['category_name']}] ({row['stock_status']})")
    
    # 3. Subquery example
    print("\n3. Products above average price in their category:")
    cursor.execute("""
        SELECT p.name, p.price, c.name as category,
               (SELECT AVG(price) FROM products WHERE category_id = p.category_id) as avg_category_price
        FROM products p
        JOIN categories c ON p.category_id = c.id
        WHERE p.price > (
            SELECT AVG(price) 
            FROM products 
            WHERE category_id = p.category_id
        )
        ORDER BY p.price DESC
    """)
    
    for row in cursor.fetchall():
        print(f"  {row['name']} - ${row['price']:.2f} (avg: ${row['avg_category_price']:.2f})")
    
    # 4. JSON queries
    print("\n4. Customer addresses (JSON extraction):")
    cursor.execute("""
        SELECT first_name || ' ' || last_name as name,
               json_extract(address, '$.city') as city,
               json_extract(address, '$.state') as state
        FROM customers
        WHERE address IS NOT NULL
        ORDER BY json_extract(address, '$.state')
    """)
    
    for row in cursor.fetchall():
        print(f"  {row['name']} - {row['city']}, {row['state']}")
    
    # 5. Aggregation queries
    print("\n5. Category statistics:")
    cursor.execute("""
        SELECT c.name as category,
               COUNT(p.id) as product_count,
               AVG(p.price) as avg_price,
               MIN(p.price) as min_price,
               MAX(p.price) as max_price,
               SUM(p.stock_quantity) as total_stock
        FROM categories c
        LEFT JOIN products p ON c.id = p.category_id AND p.is_active = 1
        WHERE c.parent_id IS NOT NULL  -- Only subcategories
        GROUP BY c.id, c.name
        HAVING COUNT(p.id) > 0
        ORDER BY avg_price DESC
    """)
    
    print(f"{'Category':<20} {'Count':<8} {'Avg Price':<12} {'Min':<10} {'Max':<12} {'Stock':<8}")
    print("-" * 75)
    for row in cursor.fetchall():
        print(f"{row['category']:<20} {row['product_count']:<8} "
              f"${row['avg_price']:<11.2f} ${row['min_price']:<9.2f} "
              f"${row['max_price']:<11.2f} {row['total_stock']:<8}")

demonstrate_select_patterns()
print("\n✅ Advanced SELECT patterns demonstrated")

## Section 4: INSERT with Orders - Complex Data Entry

In [ ]:
# Exercise 4.1: Complex order creation with multiple INSERTs

def create_sample_orders():
    """Create sample orders with order items"""
    
    cursor = conn.cursor()
    
    # Order 1: John Doe's electronics order
    order_data = {
        'customer_id': 1,  # John Doe
        'items': [
            {'product_id': 1, 'quantity': 1},  # Gaming Laptop
            {'product_id': 3, 'quantity': 2}   # SmartPhone X
        ]
    }
    
    order_id = process_order(cursor, order_data)
    print(f"✅ Created order {order_id} for John Doe")
    
    # Order 2: Jane Smith's mixed order
    order_data = {
        'customer_id': 2,  # Jane Smith
        'items': [
            {'product_id': 2, 'quantity': 1},  # Workstation Desktop
            {'product_id': 4, 'quantity': 3},  # Mystery Novel
            {'product_id': 5, 'quantity': 1}   # Python Programming Guide
        ]
    }
    
    order_id = process_order(cursor, order_data)
    print(f"✅ Created order {order_id} for Jane Smith")
    
    conn.commit()

def process_order(cursor, order_data):
    """Process a complete order with validation"""
    
    try:
        cursor.execute("BEGIN TRANSACTION")
        
        # Generate order number
        order_number = f"ORD-{int(time.time())}-{order_data['customer_id']}"
        
        # Create order record
        cursor.execute("""
            INSERT INTO orders (order_number, customer_id, subtotal, tax_amount, shipping_amount)
            VALUES (?, ?, 0, 0, 0)
        """, (order_number, order_data['customer_id']))
        
        order_id = cursor.lastrowid
        subtotal = 0
        
        # Process each order item
        for item in order_data['items']:
            # Get product price and check stock
            cursor.execute("""
                SELECT price, stock_quantity FROM products 
                WHERE id = ? AND is_active = 1
            """, (item['product_id'],))
            
            product = cursor.fetchone()
            if not product:
                raise ValueError(f"Product {item['product_id']} not found or inactive")
            
            if product['stock_quantity'] < item['quantity']:
                raise ValueError(f"Insufficient stock for product {item['product_id']}")
            
            unit_price = product['price']
            line_total = unit_price * item['quantity']
            subtotal += line_total
            
            # Create order item
            cursor.execute("""
                INSERT INTO order_items (order_id, product_id, quantity, unit_price)
                VALUES (?, ?, ?, ?)
            """, (order_id, item['product_id'], item['quantity'], unit_price))
            
            # Update product stock
            cursor.execute("""
                UPDATE products 
                SET stock_quantity = stock_quantity - ?,
                    updated_at = CURRENT_TIMESTAMP
                WHERE id = ?
            """, (item['quantity'], item['product_id']))
        
        # Calculate tax and shipping
        tax_rate = 0.08  # 8% tax
        tax_amount = subtotal * tax_rate
        
        shipping_amount = 0 if subtotal > 1000 else 15.00  # Free shipping over $1000
        
        # Update order totals
        cursor.execute("""
            UPDATE orders 
            SET subtotal = ?, tax_amount = ?, shipping_amount = ?,
                status = 'confirmed', updated_at = CURRENT_TIMESTAMP
            WHERE id = ?
        """, (subtotal, tax_amount, shipping_amount, order_id))
        
        cursor.execute("COMMIT")
        return order_id
        
    except Exception as e:
        cursor.execute("ROLLBACK")
        print(f"❌ Order processing failed: {e}")
        raise

create_sample_orders()
print("\n✅ Sample orders created successfully")

## Section 5: UPDATE - Data Modification Mastery

In [ ]:
# Exercise 5.1: Various UPDATE patterns

def demonstrate_update_patterns():
    """Demonstrate different UPDATE patterns and techniques"""
    
    cursor = conn.cursor()
    
    print("UPDATE Operations Demonstration:")
    print("=" * 40)
    
    # 1. Simple UPDATE with conditions
    print("\n1. Updating product prices (10% increase for electronics):")
    cursor.execute("""
        UPDATE products 
        SET price = ROUND(price * 1.10, 2),
            updated_at = CURRENT_TIMESTAMP
        WHERE category_id IN (6, 7)  -- Computers and Smartphones
    """)
    print(f"   Updated {cursor.rowcount} products")
    
    # 2. Conditional UPDATE with CASE
    print("\n2. Updating customer types based on credit limit:")
    cursor.execute("""
        UPDATE customers 
        SET customer_type = CASE 
            WHEN credit_limit >= 10000 THEN 'vip'
            WHEN credit_limit >= 5000 THEN 'premium'
            ELSE 'regular'
        END,
        updated_at = CURRENT_TIMESTAMP
        WHERE customer_type != CASE 
            WHEN credit_limit >= 10000 THEN 'vip'
            WHEN credit_limit >= 5000 THEN 'premium'
            ELSE 'regular'
        END
    """)
    print(f"   Updated {cursor.rowcount} customers")
    
    # 3. UPDATE with JOIN (using subquery in SQLite)
    print("\n3. Updating order status to 'processing' for confirmed orders:")
    cursor.execute("""
        UPDATE orders 
        SET status = 'processing',
            updated_at = CURRENT_TIMESTAMP
        WHERE status = 'confirmed' 
        AND id IN (
            SELECT o.id FROM orders o
            JOIN customers c ON o.customer_id = c.id
            WHERE c.customer_type IN ('premium', 'vip')
        )
    """)
    print(f"   Updated {cursor.rowcount} orders")
    
    # 4. JSON field UPDATE
    print("\n4. Adding loyalty points to customer address data:")
    cursor.execute("""
        UPDATE customers 
        SET address = json_set(
            COALESCE(address, '{}'), 
            '$.loyalty_points', 
            CASE customer_type 
                WHEN 'vip' THEN 1000
                WHEN 'premium' THEN 500
                ELSE 100
            END
        ),
        updated_at = CURRENT_TIMESTAMP
    """)
    print(f"   Updated {cursor.rowcount} customer records")
    
    # 5. Bulk UPDATE with constraints
    print("\n5. Bulk updating stock levels (restocking):")
    restock_data = [
        (20, 1),  # Laptop stock +20
        (10, 2),  # Desktop stock +10
        (50, 3),  # Phone stock +50
        (25, 4),  # Book stock +25
        (15, 5)   # Python book stock +15
    ]
    
    cursor.executemany("""
        UPDATE products 
        SET stock_quantity = stock_quantity + ?,
            updated_at = CURRENT_TIMESTAMP
        WHERE id = ?
    """, restock_data)
    print(f"   Restocked {len(restock_data)} products")
    
    conn.commit()
    
    # Verify updates
    print("\n6. Verification - Current product prices and stock:")
    cursor.execute("""
        SELECT p.name, p.price, p.stock_quantity, c.name as category
        FROM products p
        JOIN categories c ON p.category_id = c.id
        ORDER BY c.name, p.name
    """)
    
    for row in cursor.fetchall():
        print(f"   {row['name']} - ${row['price']:.2f} (Stock: {row['stock_quantity']}) [{row['category']}]")

demonstrate_update_patterns()
print("\n✅ UPDATE patterns demonstrated successfully")

## Section 6: DELETE - Safe Data Removal

In [ ]:
# Exercise 6.1: Safe DELETE operations

def demonstrate_delete_patterns():
    """Demonstrate safe DELETE patterns with validation"""
    
    cursor = conn.cursor()
    
    print("DELETE Operations Demonstration:")
    print("=" * 40)
    
    # First, let's add some test data that can be safely deleted
    print("\n1. Adding test data for deletion demos:")
    
    # Add inactive products
    test_products = [
        ('TEST-001', 'Test Product 1', 'Test product for deletion', 6, 99.99, 50.00, 0, 0, 0.1, '{}', '{}', 0),
        ('TEST-002', 'Test Product 2', 'Another test product', 7, 199.99, 100.00, 0, 0, 0.2, '{}', '{}', 0)
    ]
    
    cursor.executemany("""
        INSERT INTO products (sku, name, description, category_id, price, cost, 
                            stock_quantity, reorder_level, weight, dimensions, specifications, is_active)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, test_products)
    
    # Add inactive customer
    cursor.execute("""
        INSERT INTO customers (first_name, last_name, email, phone, customer_type, is_active)
        VALUES ('Test', 'User', 'test@example.com', '+1-555-9999', 'regular', 0)
    """)
    
    print(f"   Added {len(test_products)} test products and 1 test customer")
    
    # 2. Safe DELETE with conditions
    print("\n2. Deleting inactive products with zero stock:")
    cursor.execute("""
        DELETE FROM products 
        WHERE is_active = 0 
        AND stock_quantity = 0
        AND id NOT IN (SELECT DISTINCT product_id FROM order_items)
    """)
    print(f"   Deleted {cursor.rowcount} inactive products")
    
    # 3. Conditional DELETE with validation
    print("\n3. Attempting to delete customers without orders:")
    
    # First check what would be deleted
    cursor.execute("""
        SELECT c.id, c.first_name, c.last_name, c.email,
               (SELECT COUNT(*) FROM orders WHERE customer_id = c.id) as order_count
        FROM customers c
        WHERE c.is_active = 0
    """)
    
    inactive_customers = cursor.fetchall()
    print(f"   Found {len(inactive_customers)} inactive customers:")
    
    for customer in inactive_customers:
        print(f"     {customer['first_name']} {customer['last_name']} - Orders: {customer['order_count']}")
    
    # Safe delete only customers without orders
    cursor.execute("""
        DELETE FROM customers 
        WHERE is_active = 0 
        AND id NOT IN (SELECT DISTINCT customer_id FROM orders)
    """)
    print(f"   Safely deleted {cursor.rowcount} customers")
    
    # 4. DELETE with backup (simulation)
    print("\n4. Creating backup before potential deletion:")
    
    # Create backup table
    cursor.execute("""
        CREATE TEMP TABLE products_backup AS 
        SELECT * FROM products WHERE stock_quantity = 0
    """)
    
    # Check backup
    cursor.execute("SELECT COUNT(*) FROM products_backup")
    backup_count = cursor.fetchone()[0]
    print(f"   Backed up {backup_count} products with zero stock")
    
    # 5. Age-based DELETE (cleanup old data)
    print("\n5. Simulating cleanup of old temporary data:")
    
    # Delete any remaining test data
    cursor.execute("""
        DELETE FROM products 
        WHERE sku LIKE 'TEST-%'
    """)
    print(f"   Cleaned up {cursor.rowcount} test products")
    
    conn.commit()
    
    # 6. Verify current state
    print("\n6. Current database state after deletions:")
    
    tables = ['customers', 'products', 'orders', 'order_items']
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        print(f"   {table}: {count} records")

def safe_delete_with_dependencies(cursor, table, record_id, dependency_checks):
    """Generic function for safe deletion with dependency validation"""
    
    # Check dependencies
    for check in dependency_checks:
        cursor.execute(check['query'], (record_id,))
        count = cursor.fetchone()[0]
        
        if count > 0:
            raise ValueError(f"Cannot delete: {count} {check['description']} exist")
    
    # Safe to delete
    cursor.execute(f"DELETE FROM {table} WHERE id = ?", (record_id,))
    return cursor.rowcount

demonstrate_delete_patterns()
print("\n✅ DELETE patterns demonstrated safely")

## Section 7: DROP TABLE - Schema Management

In [ ]:
# Exercise 7.1: Safe table dropping with dependency management

def demonstrate_drop_table_patterns():
    """Demonstrate safe DROP TABLE operations"""
    
    cursor = conn.cursor()
    
    print("DROP TABLE Operations Demonstration:")
    print("=" * 45)
    
    # 1. Create temporary tables for demonstration
    print("\n1. Creating temporary tables for DROP demonstration:")
    
    temp_tables = [
        ("demo_table_1", """
            CREATE TEMP TABLE demo_table_1 (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """),
        ("demo_table_2", """
            CREATE TEMP TABLE demo_table_2 (
                id INTEGER PRIMARY KEY,
                demo1_id INTEGER REFERENCES demo_table_1(id),
                description TEXT
            )
        """)
    ]
    
    for table_name, create_sql in temp_tables:
        cursor.execute(create_sql)
        print(f"   Created {table_name}")
    
    # Add some data
    cursor.execute("INSERT INTO demo_table_1 (name) VALUES ('Test Record')")
    cursor.execute("INSERT INTO demo_table_2 (demo1_id, description) VALUES (1, 'Related Record')")
    
    # 2. Attempt to drop table with dependencies
    print("\n2. Attempting to drop table with foreign key dependencies:")
    
    try:
        # This should be safe since it's a temp table, but let's check dependencies first
        dependency_exists = check_foreign_key_dependencies(cursor, 'demo_table_1')
        
        if dependency_exists:
            print("   ⚠️  Table has dependencies, checking reference count...")
            
            cursor.execute("SELECT COUNT(*) FROM demo_table_2 WHERE demo1_id IS NOT NULL")
            ref_count = cursor.fetchone()[0]
            print(f"   Found {ref_count} referencing records")
        
        # For demo purposes, we'll drop the dependent table first
        cursor.execute("DROP TABLE IF EXISTS demo_table_2")
        print("   Dropped dependent table demo_table_2")
        
        cursor.execute("DROP TABLE IF EXISTS demo_table_1")
        print("   Dropped demo_table_1")
        
    except Exception as e:
        print(f"   ❌ DROP failed: {e}")
    
    # 3. Create and drop with backup
    print("\n3. Creating table with backup before DROP:")
    
    # Create a new demo table
    cursor.execute("""
        CREATE TEMP TABLE backup_demo (
            id INTEGER PRIMARY KEY,
            data TEXT,
            value REAL
        )
    """)
    
    # Add sample data
    sample_data = [(f"Record {i}", i * 1.5) for i in range(1, 6)]
    cursor.executemany("INSERT INTO backup_demo (data, value) VALUES (?, ?)", sample_data)
    
    print(f"   Created backup_demo table with {len(sample_data)} records")
    
    # Create backup before dropping
    backup_table_name = f"backup_demo_archive_{int(time.time())}"
    cursor.execute(f"CREATE TEMP TABLE {backup_table_name} AS SELECT * FROM backup_demo")
    
    # Verify backup
    cursor.execute(f"SELECT COUNT(*) FROM {backup_table_name}")
    backup_count = cursor.fetchone()[0]
    print(f"   Created backup table {backup_table_name} with {backup_count} records")
    
    # Now safe to drop original
    cursor.execute("DROP TABLE backup_demo")
    print("   Dropped original backup_demo table")
    
    # 4. Schema cleanup - drop unused indexes
    print("\n4. Cleaning up unused database objects:")
    
    # Check for any temporary indexes
    cursor.execute("""
        SELECT name, type FROM sqlite_master 
        WHERE type = 'index' 
        AND name NOT LIKE 'sqlite_%'
        AND name LIKE '%temp%'
    """)
    
    temp_indexes = cursor.fetchall()
    print(f"   Found {len(temp_indexes)} temporary indexes to potentially clean")
    
    # 5. Verify current schema state
    print("\n5. Current schema state:")
    
    cursor.execute("""
        SELECT type, COUNT(*) as count
        FROM sqlite_master 
        WHERE name NOT LIKE 'sqlite_%'
        GROUP BY type
        ORDER BY type
    """)
    
    for row in cursor.fetchall():
        print(f"   {row['type']}s: {row['count']}")
    
    # List all tables
    cursor.execute("""
        SELECT name FROM sqlite_master 
        WHERE type = 'table' AND name NOT LIKE 'sqlite_%'
        ORDER BY name
    """)
    
    tables = [row['name'] for row in cursor.fetchall()]
    print(f"   Active tables: {', '.join(tables)}")
    
    conn.commit()

def check_foreign_key_dependencies(cursor, table_name):
    """Check if a table has foreign key dependencies"""
    
    cursor.execute("""
        SELECT sql FROM sqlite_master 
        WHERE type = 'table' 
        AND sql LIKE '%REFERENCES ' || ? || '%'
    """, (table_name,))
    
    return len(cursor.fetchall()) > 0

def safe_drop_table_with_backup(cursor, table_name):
    """Safely drop a table with automatic backup creation"""
    
    # Create backup
    backup_name = f"{table_name}_backup_{int(time.time())}"
    cursor.execute(f"CREATE TABLE {backup_name} AS SELECT * FROM {table_name}")
    
    # Verify backup
    cursor.execute(f"SELECT COUNT(*) FROM {backup_name}")
    backup_count = cursor.fetchone()[0]
    
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    original_count = cursor.fetchone()[0]
    
    if backup_count != original_count:
        raise ValueError("Backup verification failed")
    
    # Safe to drop
    cursor.execute(f"DROP TABLE {table_name}")
    
    return backup_name

demonstrate_drop_table_patterns()
print("\n✅ DROP TABLE patterns demonstrated safely")

## Section 8: Performance Analysis and Optimization

In [ ]:
# Exercise 8.1: Query performance analysis for Big-6 operations

def analyze_big_six_performance():
    """Analyze performance of all Big-6 SQL operations"""
    
    cursor = conn.cursor()
    
    print("Big-6 SQL Performance Analysis:")
    print("=" * 45)
    
    # Performance test queries
    test_queries = [
        ("Complex SELECT with JOINs", """
            SELECT c.first_name, c.last_name, 
                   COUNT(o.id) as order_count,
                   SUM(o.total_amount) as total_spent,
                   AVG(o.total_amount) as avg_order
            FROM customers c
            LEFT JOIN orders o ON c.id = o.customer_id
            WHERE c.is_active = 1
            GROUP BY c.id, c.first_name, c.last_name
            HAVING COUNT(o.id) > 0
            ORDER BY total_spent DESC
        """),
        
        ("Product search with category", """
            SELECT p.name, p.price, c.name as category,
                   p.stock_quantity,
                   json_extract(p.specifications, '$.brand') as brand
            FROM products p
            JOIN categories c ON p.category_id = c.id
            WHERE p.is_active = 1
            AND p.stock_quantity > 0
            AND p.price BETWEEN 50 AND 2000
            ORDER BY p.price DESC
        """),
        
        ("Order details with items", """
            SELECT o.order_number, o.order_date, o.total_amount,
                   c.first_name || ' ' || c.last_name as customer,
                   COUNT(oi.id) as item_count,
                   GROUP_CONCAT(p.name) as products
            FROM orders o
            JOIN customers c ON o.customer_id = c.id
            JOIN order_items oi ON o.id = oi.order_id
            JOIN products p ON oi.product_id = p.id
            GROUP BY o.id, o.order_number, o.order_date, o.total_amount, customer
            ORDER BY o.order_date DESC
        """)
    ]
    
    for query_name, query in test_queries:
        print(f"\n📊 {query_name}:")
        
        # Analyze query plan
        cursor.execute(f"EXPLAIN QUERY PLAN {query}")
        plan = cursor.fetchall()
        
        print("   Query Plan:")
        for row in plan:
            print(f"     {row[0]}|{row[1]}|{row[2]}|{row[3]}")
        
        # Time the query
        start_time = time.time()
        cursor.execute(query)
        results = cursor.fetchall()
        execution_time = time.time() - start_time
        
        print(f"   Execution time: {execution_time:.4f} seconds")
        print(f"   Results returned: {len(results)}")
        
        # Show sample results
        if results:
            print(f"   Sample result: {dict(results[0])}")
    
    # Database statistics
    print("\n📈 Database Statistics:")
    
    # Table sizes
    tables = ['customers', 'products', 'categories', 'orders', 'order_items']
    total_records = 0
    
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        total_records += count
        print(f"   {table}: {count:,} records")
    
    print(f"   Total records: {total_records:,}")
    
    # Index usage
    cursor.execute("""
        SELECT COUNT(*) as index_count
        FROM sqlite_master 
        WHERE type = 'index' AND name NOT LIKE 'sqlite_%'
    """)
    index_count = cursor.fetchone()[0]
    print(f"   Custom indexes: {index_count}")
    
    # Database file size (approximation)
    cursor.execute("PRAGMA page_count")
    page_count = cursor.fetchone()[0]
    
    cursor.execute("PRAGMA page_size")
    page_size = cursor.fetchone()[0]
    
    file_size_mb = (page_count * page_size) / (1024 * 1024)
    print(f"   Database size: ~{file_size_mb:.2f} MB")

analyze_big_six_performance()
print("\n✅ Performance analysis completed")

## Section 9: Lab Summary and Best Practices

In [ ]:
# Final summary and best practices demonstration

def generate_big_six_summary():
    """Generate comprehensive summary of Big-6 mastery"""
    
    cursor = conn.cursor()
    
    print("=" * 60)
    print("BIG-6 SQL STATEMENTS MASTERY SUMMARY")
    print("=" * 60)
    
    # Database state summary
    cursor.execute("""
        SELECT 'customers' as table_name, COUNT(*) as records FROM customers
        UNION ALL
        SELECT 'products', COUNT(*) FROM products
        UNION ALL  
        SELECT 'categories', COUNT(*) FROM categories
        UNION ALL
        SELECT 'orders', COUNT(*) FROM orders
        UNION ALL
        SELECT 'order_items', COUNT(*) FROM order_items
    """)
    
    print("\n📊 Final Database State:")
    total_records = 0
    for row in cursor.fetchall():
        print(f"  {row[0]}: {row[1]:,} records")
        total_records += row[1]
    print(f"  Total: {total_records:,} records")
    
    # Business metrics
    cursor.execute("""
        SELECT 
            COUNT(DISTINCT customer_id) as active_customers,
            COUNT(*) as total_orders,
            SUM(total_amount) as total_revenue,
            AVG(total_amount) as avg_order_value
        FROM orders
        WHERE status NOT IN ('cancelled')
    """)
    
    metrics = cursor.fetchone()
    print("\n💰 Business Metrics:")
    print(f"  Active customers: {metrics[0]}")
    print(f"  Total orders: {metrics[1]}")
    print(f"  Total revenue: ${metrics[2]:,.2f}")
    print(f"  Average order value: ${metrics[3]:,.2f}")
    
    # Technical achievements
    print("\n✅ Big-6 SQL Mastery Achieved:")
    achievements = [
        "CREATE TABLE: Comprehensive schema with constraints",
        "INSERT: Single, bulk, and complex data insertion",
        "SELECT: Advanced queries with JOINs, subqueries, JSON",
        "UPDATE: Conditional, bulk, and JSON field updates",
        "DELETE: Safe deletion with dependency validation",
        "DROP TABLE: Schema cleanup with backup strategies"
    ]
    
    for achievement in achievements:
        print(f"  ✓ {achievement}")
    
    # Best practices demonstrated
    print("\n🎯 Best Practices Demonstrated:")
    practices = [
        "Parameterized queries for SQL injection prevention",
        "Transaction management for data consistency",
        "Foreign key constraints for referential integrity",
        "Check constraints for data validation",
        "Indexes for query performance optimization",
        "JSON storage for flexible data structures",
        "Error handling and validation patterns",
        "Performance analysis with EXPLAIN QUERY PLAN"
    ]
    
    for practice in practices:
        print(f"  ✓ {practice}")

# Generate final summary
generate_big_six_summary()

# Close database connection
conn.close()

print("\n" + "=" * 60)
print("🎉 MODULE 2 COMPLETED - BIG-6 SQL MASTERY ACHIEVED!")
print("=" * 60)
print("\nYou have successfully mastered:")
print("• All six essential SQL statements")
print("• Advanced query patterns and techniques")
print("• Safe data manipulation strategies")
print("• Performance optimization methods")
print("• Professional best practices")
print("\nReady for Module 3: Aggregate Functions & Analytics! 📊")